In [1]:
import pandas as pd
import json

In [2]:

with open('../variables/factores.json') as archivo_factores:
    datos_factores_json=json.load(archivo_factores)
datos_factores=pd.DataFrame(datos_factores_json)


with open('../input/informacion_energetica.json') as archivo_info_energetica:
    datos_info_energetica_json=json.load(archivo_info_energetica)
datos_info_energetica=pd.DataFrame(datos_info_energetica_json)

In [3]:
datos_factores


,Potencia kWAC,Combinacion de inversores,Pdc max (kWp)
0,40,GW20KT-MT + GW20KT-MT,50.0
1,50,GW50KN-MT,62.5
2,210,GW80K-MT + GW80K-MT + GW50KN-MT,262.5


In [4]:
potencia_peak=260 #Input
caracteristica_panel = 540  #Input ? 


In [5]:

potencia_deseada=potencia_peak
panel_utilizado=caracteristica_panel

cantidad_paneles=round((potencia_deseada*1000/panel_utilizado))

potencia_calculada= (panel_utilizado*cantidad_paneles)/1000

costo_mantencion_por_vez=840987 #Datos de regiones
costo_mantecion_total=3363948 #Datos de regiones

costo_administracion_anual=200000 #Input?
uf_hoy = 34326 #Data

costo_anual_om=costo_administracion_anual + costo_mantecion_total


costos_2022=[]
costos_2022.append ({
    "Potencia deseada": potencia_peak,
    "Panel Utilizado" : caracteristica_panel,
    "Cantidad de Paneles":cantidad_paneles,
    "Potencia calculada":potencia_calculada,
    "Costo mantencion por vez":  costo_mantencion_por_vez,
    "Costo mantencion total": costo_mantecion_total,
    "Costo de Administracion anual": costo_administracion_anual,
    "UF ": uf_hoy,
    "Costo Total Anual O&M": costo_anual_om

})
with open('../variables/costos2022.json', 'w') as file:
    json.dump(costos_2022, file, indent=4)


In [6]:
with open('../variables/costos2022.json') as archivo_costos:
    datos_costos_json=json.load(archivo_costos)
datos_costos=pd.DataFrame(datos_costos_json)
datos_costos

,Potencia deseada,Panel Utilizado,Cantidad de Paneles,Potencia calculada,Costo mantencion por vez,Costo mantencion total,Costo de Administracion anual,UF,Costo Total Anual O&M
0,260,540,481,259.74,840987,3363948,200000,34326,3563948


In [7]:
#Datos Factores


#POTENCIA INVERSORES
pontencia_inversores=datos_factores["Pdc max (kWp)"]
contador=0
potencia_peak=160
for a in pontencia_inversores:
    potencia_inversor = a

    if contador == 0:
        if potencia_peak <=a:
            inversor = a
            break
        
        potencia_inversor_anterior=a
    else:
        if potencia_peak <= a and potencia_peak > potencia_inversor_anterior:
            inversor = a

            break
        else:
            potencia_inversor_anterior=a
    contador +=1


pdc_max= inversor

for a in datos_factores:
    combina_inversores=datos_factores["Combinacion de inversores"][contador]
    pontencia_kWac=datos_factores["Potencia kWAC"][contador]

    

combina_inversores,inversor,pontencia_kWac
pontencia_kWac=int(pontencia_kWac)


In [8]:
sobredimensionamiento= (potencia_calculada/pontencia_kWac) * 100
sobredimensionamiento 

123.68571428571428

## Generación especifica y generación anual


In [12]:
factor_coplanar=0.950
aumento_generacion=1.045
pes_correj_flux=1775


generacion_especifica=factor_coplanar * aumento_generacion * pes_correj_flux

generacion_anual= generacion_especifica*potencia_calculada


generacion_especifica, generacion_anual


(1762.13125, 457695.970875)

In [10]:
kwh_total=datos_info_energetica.sum(axis = 0, skipna = True)["kWh/Mes"]
kwh_total

balance_generacion_consumo=kwh_total*generacion_anual

categoria_tipo_montaje="Techo coplanar"

suelo=13
techo_estructurado=13
techo_coplanar=9


if categoria_tipo_montaje== "Suelo":
    
    superf_max_paneles= potencia_calculada * suelo #Suelos desde DB
elif categoria_tipo_montaje == "Techo estructurado":

    superf_max_paneles= potencia_calculada * techo_estructurado #Suelos desde DB
else:
    superf_max_paneles= potencia_calculada * techo_coplanar #Suelos desde DB



costo_planta_solar=(potencia_calculada*1000) * 621.8 #Variable desde DB
adicionales=22000000

costo_total=costo_planta_solar + adicionales


## Caracteristicas de instalación


In [11]:
caracteristicas_instalacion=[]

caracteristicas_instalacion.append ({
    "Potencia peak deseada": potencia_peak,
    "Sobredimensionamiento" : sobredimensionamiento,
    "Potencia inversores":pontencia_kWac,
    "Combinacion de inversores":inversor,
    "Potencia de panel":  0,
    "Cantidad de paneles": cantidad_paneles,
    "Potencia peak": potencia_calculada,
    "Comuna instalacion": "Alto del Carmen",
    "Region/Zona": "Region de Atacama" ,
    "Generacion especifica":generacion_especifica ,
    "Generacion inicial total":generacion_anual,
    "Balance generacion/consumo":balance_generacion_consumo,
    "Categoria Zona":"Region Atacama",
    "Categoria Tipo Montaje*":categoria_tipo_montaje,
    "Superficie max. paneles":superf_max_paneles,
    "Costo anual por O&M":costo_anual_om,
    "Costo Planta Solar":costo_planta_solar,
    "Costos Adicionales*":adicionales,
    "Costo total": costo_total

})
with open('../variables/caracteristicas_instalacion.json', 'w') as file:
    json.dump(caracteristicas_instalacion, file, indent=4)


with open('../variables/caracteristicas_instalacion.json') as file:
    datos_info_energetica=json.load(file)
data_info=pd.DataFrame(datos_info_energetica)

data_info

,Potencia peak deseada,Sobredimensionamiento,Potencia inversores,Combinacion de inversores,Potencia de panel,Cantidad de paneles,Potencia peak,Comuna instalacion,Region/Zona,Generacion especifica,Generacion inicial total,Balance generacion/consumo,Categoria Zona,Categoria Tipo Montaje*,Superficie max. paneles,Costo anual por O&M,Costo Planta Solar,Costos Adicionales*,Costo total
0,160,123.685714,210,262.5,0,481,259.74,Alto del Carmen,Region de Atacama,1762.13125,457695.970875,1.696482e+11,Region Atacama,Techo coplanar,2337.66,3563948,161506332.0,22000000,183506332.0
